In [7]:
import pandas as pd
import numpy as np 
import datetime
inventory = pd.read_csv("data/inventory.csv")
products = pd.read_csv("data/products_ext.csv")
promotions = pd.read_csv("data/promotions.csv")
transactions = pd.read_csv("data/transactions.csv")

#create a reference dataframe
dayofyear = list(range(365))
date = [datetime.datetime(2018, 1, 1) + datetime.timedelta(day) for day in dayofyear]
df_date = pd.DataFrame({"day":dayofyear, "date":date})

import ast
for colname in inventory.columns[2:]:
    inventory[colname] = [ast.literal_eval(i) for i in inventory[colname]]
    #converting inventory rows to list of tuples from string

In [8]:
products.head()

,category,product_id,description,size,std_sales_price,bio,basic,purchase_price
0,vegetable,v_1,Komkommer,1st,0.69,0,0,0.18
1,vegetable,v_2,Courgette,1st,0.75,0,0,0.17
2,vegetable,v_3,SnoepVegetable tomaat,500g,1.99,0,0,0.50
3,vegetable,v_4,Paprika Mix,3st,1.79,0,0,0.40
4,vegetable,v_5,Paprika,1st,0.99,0,0,0.23


In [82]:
inventory[['Blauwe bessen.1']]

,Blauwe bessen.1
0,"[(20, 50)]"
1,"[(20, 50)]"
2,"[(20, 14), (23, 50)]"
3,"[(20, 14), (23, 50)]"
4,"[(23, 32), (27, 50)]"
...,...
203,"[(356, 36), (359, 50), (363, 10), (366, 10), (..."
204,"[(359, 43), (363, 10), (366, 10), (370, 50), (..."
205,"[(359, 43), (363, 10), (366, 10), (370, 50), (..."
206,"[(363, 3), (366, 10), (370, 50), (373, 50), (3..."


In [120]:
product = 'Biologische spekreepjes' #select the product to analyze
transactions[transactions["description"] == product] #transactions for that product

,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
1782,2018-02-01,18:44:22,101.0,24107630.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
1827,2018-02-01,18:47:38,102.0,34187107.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
2103,2018-03-01,10:47:17,117.0,1064747.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
2724,2018-03-01,19:25:33,158.0,63634047.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
4534,2018-06-01,12:45:08,274.0,43670698.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
353828,2018-12-29,20:04:32,20636.0,38621378.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
354041,2018-12-30,10:08:47,20646.0,NaN,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
354144,2018-12-30,12:42:05,20650.0,63634047.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0
354171,2018-12-30,13:15:05,20651.0,79747334.0,meat,m_15,Biologische spekreepjes,200g,2.69,2.69,1.0,0.0


In [121]:
def date_converter(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["day"])
    return converted_date

inventory["date"] = inventory.apply(date_converter, axis = 1) #convert day of year to actual timestamp

In [107]:
def week_of_year(row):
    week_nr = datetime.date(row["date"].year, row["date"].month, row["date"].day).isocalendar()[1]
    return week_nr

inventory["week"] = inventory.apply(week_of_year, axis = 1)
#add nr week of year as an extra column to inventory

In [108]:
#only keep relevant columns in inventory
filter_inventory = inventory[["day", "before or after delivery", "date", "week", product]]
#filters promotion table to only contain discounts for chosen product
promotions_filter = promotions[promotions["description"] == product]

def check_discount(row):
    if row["week"] in promotions_filter["week"].values:
        discount_amount = promotions_filter[promotions_filter["week"] == row["week"]]["discount"]
        return discount_amount.values[0]
    else:
        return 0
#adds extra column to filter inventory containing discount amount
filter_inventory["discount"] = filter_inventory.apply(check_discount, axis = 1)


ipykernel_launcher:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [109]:
def total_inv(row):
    total = 0
    if len(row[product]) > 0:
        for batch in row[product]:
            total += batch[1]
    return total
#adds total nr items in inventory for the filtered dataframe
filter_inventory["total inventory"] = filter_inventory.apply(total_inv, axis = 1)

ipykernel_launcher:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [110]:
best_before = []
for element_list in filter_inventory[product]:
    for batch in element_list:
        best_before.append(batch[0])
bb_dates = np.unique(best_before)
#list of each unique best before dates
bb_dates

array([ 10,  13,  17,  20,  24,  27,  31,  34,  38,  41,  45,  48,  52,
        55,  59,  62,  66,  69,  73,  76,  80,  83,  87,  90,  94,  97,
       101, 104, 108, 111, 115, 118, 122, 125, 129, 132, 136, 139, 143,
       146, 150, 153, 157, 160, 164, 167, 171, 174, 178, 181, 185, 188,
       192, 195, 199, 202, 206, 209, 213, 216, 220, 223, 227, 230, 234,
       237, 241, 244, 248, 251, 255, 258, 262, 265, 269, 272, 276, 279,
       283, 286, 290, 293, 297, 300, 304, 307, 311, 314, 318, 321, 325,
       328, 332, 335, 339, 342, 346, 349, 353, 356, 360, 363, 367, 370])

In [111]:
df_waste = pd.DataFrame({"best before":bb_dates})
df_waste #build initial df with points where waste can occur

,best before
0,10
1,13
2,17
3,20
4,24
...,...
99,356
100,360
101,363
102,367


In [112]:
def input_inv(row):
    for element_list in filter_inventory[product]:
        if len(element_list) > 0:
            for batch in element_list:
                if batch[0] == row["best before"]:
                    return batch[1]

df_waste["amount"] = df_waste.apply(input_inv, axis = 1)
df_waste #amount that was received with particular expiry date

,best before,amount
0,10,5
1,13,5
2,17,5
3,20,5
4,24,5
...,...,...
99,356,5
100,360,5
101,363,5
102,367,5


In [113]:
df_waste = df_waste[df_waste["best before"] < 365] #only keep rows where expiry date is in same year
df_waste

,best before,amount
0,10,5
1,13,5
2,17,5
3,20,5
4,24,5
...,...,...
97,349,5
98,353,5
99,356,5
100,360,5


In [114]:
transactions["day"] = pd.to_datetime(transactions["day"])
filter_transactions = transactions[transactions["description"] == product]
#converts date then filters transactions to contain selected product purchases

first_restock = df_date[df_date["day"] == 3]["date"].values[0]
#first_restock check purchases mde before the first restock
filter_transactions[filter_transactions["day"] < first_restock]
#PROBLEM: IF A PRODUCT HASN'T BEEN BOUGHT IN ONE DAY IT HAS A ZERO, CAUSES AN ERROR FEW CELLS 

,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic


In [115]:
#cumulative sum of purchases from that product
purchases_per_day = filter_transactions.groupby("day").count()["product_id"].cumsum()
#purchases made per day from that product
purchases_per_day_plotting = filter_transactions.groupby("day").count()["product_id"]
#idk know what is this
purchases_per_day_plotting.index = purchases_per_day_plotting.index.dayofyear/7
purchases_per_day
#would need to generate the individual days where there are no new purchases and assign a zero value to these days to fix the cumsum()

day
2018-01-05      1
2018-01-06      4
2018-01-07      6
2018-01-09      8
2018-01-10      9
             ... 
2018-12-26    427
2018-12-27    428
2018-12-28    429
2018-12-29    433
2018-12-30    437
Name: product_id, Length: 243, dtype: int64

In [116]:
purchases_per_day.index[0].year
weeknr = []
for ind in purchases_per_day.index:
    week = datetime.date(ind.year, ind.month, ind.day).isocalendar()[1]
    weeknr.append(week)
#week number for each purchase day


In [137]:
df_waste.drop(index=df_waste.tail(9).index, inplace=True)
#drop the last rows not to cause index issues
#STILL NEED TO BE FIXED
df_waste

/Users/teddy/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,best before,amount
0,10,5
1,13,5
2,17,5
3,20,5
4,24,5
5,27,5
6,31,5
7,34,5
8,38,5
9,41,5


In [118]:
df_waste["purchases"] = [purchases_per_day[i-1] for i in df_waste["best before"]]
#add cumulative sum of purchases up until that day (based on expiry dates)
df_waste

IndexError: index 243 is out of bounds for axis 0 with size 243

In [103]:
df_waste["prev purchases"] = df_waste["purchases"].diff()
df_waste #difference in purchases since a previous batch of product expired

ipykernel_launcher:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,best before,amount,purchases,prev purchases
0,10,7,38,NaN
1,13,7,47,9.0
2,17,4,64,17.0
3,20,4,79,15.0
4,24,16,95,16.0
...,...,...,...,...
88,318,16,1377,29.0
89,321,16,1388,11.0
90,325,16,1403,15.0
91,328,16,1420,17.0


In [104]:
length = df_waste.shape[0]
lst_waste = [] #nr rows in waste dataframe
#amount left from the item
df_waste["remaining"] = df_waste["amount"] - df_waste["prev purchases"] 
#the first row is different as there is no prev pruchases available
df_waste["remaining"].iloc[0] = df_waste["amount"].iloc[0] - df_waste["purchases"].iloc[0]

for i in range(length):
    #iterates through the rows of the waste dataframe
    current_row = df_waste.iloc[i]
    if i == 0:
        #
        lst_waste.append(current_row["remaining"])
        continue     
    else:
        prev_row = df_waste.iloc[i-1] #previous row in dataframe
        if lst_waste[-1] < 0:
            #no idea whats happening here, I've had enough
            waste = lst_waste[-1] + current_row["remaining"]
            lst_waste.append(lst_waste[-1] + current_row["remaining"])
        else:
            lst_waste.append(current_row["remaining"])
    
df_waste["waste"] = lst_waste
def date_converter(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["best before"])
    return converted_date
purchase_price = products[products['description'] == product]['purchase_price'].mean()
df_waste["date"] = df_waste.apply(date_converter, axis = 1)
df_waste["week"] = df_waste.apply(week_of_year, axis = 1)
df_waste[df_waste["waste"] < 0]["waste"] = 0

df_waste['weekPrice'] = [float(products[products['description'] == product]['std_sales_price'].mean())] * len(df_waste)
#df_waste.set_index('week', inplace = True)
#print(df_waste.head())
for week_ind in list(promotions[promotions['description'] == product].index):
    week = promotions.at[week_ind, 'week']
    print(week_ind)
    # print(df_waste[['week']].dtypes)
    if week in list(np.unique(df_waste.week)):
        print(week)
        df_waste.loc[df_waste[df_waste['week'] == week]['weekPrice'].index, 'weekPrice'] = (1 - (promotions.at[week_ind, 'discount'] / 100)) * df_waste.loc[df_waste[df_waste['week'] == week]['weekPrice'].index, 'weekPrice']
        #break
df_waste['lost_Revenue'] = df_waste.waste * df_waste.weekPrice
df_waste['purc_price'] = [float(purchase_price)] * len(df_waste)
df_waste['lost_profit'] = df_waste['waste'] * (df_waste['weekPrice'] - df_waste['purc_price'])
df_waste['cost_wasted'] = df_waste['waste'] * df_waste['purc_price']
df_waste.head(50)

13
15
2
64
5
293
20
386
26
517
35
613
41
620
42
714
48
ipykernel_launcher:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/teddy/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/teddy/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

,best before,amount,purchases,prev purchases,remaining,waste,date,week,weekPrice,lost_Revenue,purc_price,lost_profit,cost_wasted
0,10,7,38,NaN,-31.0,-31.0,2018-01-11,2,2.5555,-79.2205,0.55,-62.1705,-17.05
1,13,7,47,9.0,-2.0,-33.0,2018-01-14,2,2.5555,-84.3315,0.55,-66.1815,-18.15
2,17,4,64,17.0,-13.0,-46.0,2018-01-18,3,2.6900,-123.7400,0.55,-98.4400,-25.30
3,20,4,79,15.0,-11.0,-57.0,2018-01-21,3,2.6900,-153.3300,0.55,-121.9800,-31.35
4,24,16,95,16.0,0.0,-57.0,2018-01-25,4,2.6900,-153.3300,0.55,-121.9800,-31.35
5,27,16,116,21.0,-5.0,-62.0,2018-01-28,4,2.6900,-166.7800,0.55,-132.6800,-34.10
6,31,16,126,10.0,6.0,-56.0,2018-02-01,5,2.2865,-128.0440,0.55,-97.2440,-30.80
7,34,16,136,10.0,6.0,-50.0,2018-02-04,5,2.2865,-114.3250,0.55,-86.8250,-27.50
8,38,10,149,13.0,-3.0,-53.0,2018-02-08,6,2.6900,-142.5700,0.55,-113.4200,-29.15
9,41,10,155,6.0,4.0,-49.0,2018-02-11,6,2.6900,-131.8100,0.55,-104.8600,-26.95


In [232]:
#promotions[(promotions['description'] == product) & (promotions['week'] == week)]['discount']
#promotions[promotions['description'] == product]['week']
promotions[(promotions['description'] == product) ]

,week,category,product_id,description,discount
0,1,vegetable,v_14,Biologische knoflook,30
107,8,vegetable,v_14,Biologische knoflook,15
260,18,vegetable,v_14,Biologische knoflook,5
388,26,vegetable,v_14,Biologische knoflook,30
701,47,vegetable,v_14,Biologische knoflook,25


In [88]:
print(products[products['description'] == 'Half om half gehakt'])

   category product_id          description  size  std_sales_price  bio  \
70     meat        m_6  Half om half gehakt  500g             2.69    0   

    basic  purchase_price  
70      0            0.55  


In [32]:
df_waste[df_waste['waste']>0][['waste', 'lost_Revenue', 'lost_profit', 'cost_wasted']].sum()

waste           897.000
lost_Revenue    612.168
lost_profit     477.618
cost_wasted     134.550
dtype: float64

In [200]:
print('The number of products wasted in the dataset for {} is {} items'.format(product, df_waste[df_waste['waste']>0]['waste'].sum()))

The number of products wasted in the dataset for Biologische knoflook is 897.0 items


In [131]:
df_waste.loc[:,:].where(df_waste.loc[:,'waste']>0)

,best before,amount,purchases,prev purchases,remaining,waste,date,week
0,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,28.0,25.0,113.0,9.0,16.0,12.0,2018-01-29,5.0
...,...,...,...,...,...,...,...,...
87,318.0,25.0,1236.0,10.0,15.0,15.0,2018-11-15,46.0
88,322.0,25.0,1252.0,16.0,9.0,9.0,2018-11-19,47.0
89,325.0,25.0,1261.0,9.0,16.0,16.0,2018-11-22,47.0
90,329.0,25.0,1271.0,10.0,15.0,15.0,2018-11-26,48.0


In [132]:
df_waste[df_waste['waste']>0]['waste'].sum()

985.0

In [31]:
transactions[transactions['description'] == 'Biologische prei'].groupby('day').count().iloc[0:14, :]

,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
day,,,,,,,,,,,
2018-01-01,2,2,1,2,2,2,2,2,2,2,2
2018-01-02,1,1,0,1,1,1,1,1,1,1,1
2018-01-03,3,3,3,3,3,3,3,3,3,3,3
2018-01-04,9,9,9,9,9,9,9,9,9,9,9
2018-01-05,4,4,4,4,4,4,4,4,4,4,4
2018-01-06,3,3,1,3,3,3,3,3,3,3,3
2018-01-07,7,7,4,7,7,7,7,7,7,7,7
2018-01-08,5,5,5,5,5,5,5,5,5,5,5
2018-01-09,5,5,5,5,5,5,5,5,5,5,5


In [131]:
df_waste["waste nn"] = [i if i > 0 else 0 for i in df_waste["waste"]]
#takes nonnegative values which is counted as waste
waste_group = df_waste.groupby("week").sum()
waste_group["weekly purchases"] = waste_group["purchases"].diff()


<ipython-input-131-08904a856e7a>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## `df_waste` is prepared for plotting

In [132]:
waste_group

,best before,amount,purchases,prev purchases,remaining,waste,waste nn,weekly purchases
week,,,,,,,,
3,31,52,119,5.0,-10.0,-41.0,0.0,NaN
4,45,52,179,36.0,16.0,3.0,6.0,60.0
5,59,52,238,25.0,27.0,27.0,27.0,59.0
6,73,52,283,23.0,29.0,29.0,29.0,45.0
7,87,52,327,24.0,28.0,28.0,28.0,44.0
8,101,52,383,24.0,28.0,28.0,28.0,56.0
9,115,52,430,26.0,26.0,26.0,26.0,47.0
10,129,52,492,31.0,21.0,21.0,21.0,62.0
11,143,52,537,21.0,31.0,31.0,31.0,45.0


In [133]:
waste_group["week"] = waste_group.index

In [134]:
import plotly.express as px
fig = px.line(waste_group, x = "week", y = "waste nn", title="Absolute waste of {} per week".format(product))
fig.add_scatter(x=waste_group["week"], y=waste_group["amount"], mode="lines")
fig.add_scatter(x=waste_group["week"], y=waste_group["weekly purchases"], mode="lines")
fig.update_yaxes(title="Waste amount")
fig.show()

In [112]:
purchases_per_day

day
2018-01-01       8
2018-01-02      13
2018-01-03      18
2018-01-04      30
2018-01-05      40
              ... 
2018-12-26    2913
2018-12-27    2918
2018-12-28    2921
2018-12-29    2934
2018-12-30    2943
Name: product_id, Length: 364, dtype: int64

In [71]:
transactions["day"] = pd.to_datetime(transactions["day"])
filter_transactions = transactions[transactions["description"] == product]

In [86]:
daydate = df_date[df_date["day"] == 2]["date"].values[0]

filter_transactions[filter_transactions["day"] <= daydate]


,day,time,customer,bank acount,category,product_id,description,size,std_sales_price,purchase_price,bio,basic
89,2018-01-01,10:40:13,6.0,54211938.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
467,2018-01-01,14:21:35,26.0,51433968.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
573,2018-01-01,15:36:46,33.0,30794082.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
700,2018-01-01,16:56:20,39.0,15832817.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
808,2018-01-01,18:15:29,46.0,90399261.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
821,2018-01-01,18:15:29,46.0,90399261.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
824,2018-01-01,18:15:29,46.0,90399261.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
879,2018-01-01,18:46:48,49.0,15382238.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
29093,2018-01-02,12:52:06,1714.0,28719528.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0
29134,2018-01-02,13:44:14,1718.0,39309224.0,vegetable,v_1,Komkommer,1st,0.69,0.69,0.0,0.0


In [111]:
purchases_per_day = filter_transactions.groupby("day").count()["product_id"].cumsum()
purchases_per_day

day
2018-01-01       8
2018-01-02      13
2018-01-03      18
2018-01-04      30
2018-01-05      40
              ... 
2018-12-26    2913
2018-12-27    2918
2018-12-28    2921
2018-12-29    2934
2018-12-30    2943
Name: product_id, Length: 364, dtype: int64

In [52]:
daydate

numpy.datetime64('2018-01-15T00:00:00.000000000')

In [15]:
dayofyear = list(range(365))
date = [datetime.datetime(2018, 1, 1) + datetime.timedelta(day) for day in dayofyear]
df_date = pd.DataFrame({"day":dayofyear, "date":date})